In [148]:
import json
import re

from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.core import Annotation, Segment

In [149]:
# Calculates accurateness of Whisper ASR

# Normalizes the text by removing punctuation and converting to lowercase.
def normalize_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = text.lower()
    return text


# Calculates the Word Error Rate (WER).
def wer(reference, hypothesis):
    ref_words = normalize_text(reference).split()
    hyp_words = normalize_text(hypothesis).split()

    # Initializes matrix
    d = [[0] * (len(hyp_words) + 1) for _ in range(len(ref_words) + 1)]

    # Fills the first row and column of the matrix
    for i in range(len(ref_words) + 1):
        d[i][0] = i
    for j in range(len(hyp_words) + 1):
        d[0][j] = j

    # Populates matrix
    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(hyp_words) + 1):
            if ref_words[i - 1] == hyp_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    # The WER is the number of edits divided by the number of words in the reference
    wer_value = d[len(ref_words)][len(hyp_words)] / len(ref_words)

    return wer_value

In [199]:
# ref is whisper
# hyp is manual

# Haarlem - 2016 - 246556.mp4 - 36:07
ref = "Als er iemand stemming vraagt overigens. Als niemand stemming vraagt. Ja, wel vanuit de SP wordt dat gevraagd. Wie in de regio? Dat is dat gevraagd. Wie is voor dit beleidskader? Nou had ik beter eerst kunnen vragen wie tegen was. 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20. Ik zie de SP die stemt niet voor. Even kijken. Ja, de rest wel. Dus met stemmen van de SP tegen is het aangenomen. Dan gaan wij naar agenda punt 9. Dat is het on-eigelijk gebruik van gemeentegrond. Ik heb begrepen dat er wat punten zijn aangepast na de behandeling in de commissie. Maar dat de wethouder nog even een toevoeging heeft op die aanpassingen. De heer Bottigen. Meneer de voorzitter. Er is in de stukken een fout geslopen in het collegebesluit. Wat als raadsbesluit staat. Daar wordt wel gesproken over dat het gaat over 2,2 miljoen verdeeld over 10 jaar. Maar dat dat per jaar wordt vastgesteld. Maar in het raadsbesluit is dat verkeerd opgenomen. Dus de tekst van het stuk van het raadsbesluit. Het is zo dat eigenlijk daarmee die motie die is ingediend op dit vlak. Of het abonnement. Waarbij staat dat er gedurende het bedrag van de kadernoten van vier jaar. En dat dan per jaar wordt gekeken of dat bedrag ingezet kan worden. Dat is ook de formulering zoals het college die heeft gedaan. Alleen die is in het raadsvoorstel onvoldoende of niet verwerkt. Dus we zijn het eens met elkaar. Waarbij wel de kanttekening overigens dat het geen zin heeft om bijvoorbeeld volgend jaar al te zeggen. Van we hebben het een jaartje geprobeerd en het werkt niet"
hyp = "Als er iemand stemming vraagt overigens. Als niemand stemming vraagt. Ja Wel vanuit de SP wordt dat gevraagd. Wie in de regio? Wie is voor dit beleidskader? Nou had ik beter eerst kunnen vragen wie tegen was geloof ik. 5 6 7 8 9 10 11 12 13 14 15 16 17, 18, 19, 20. Ik zie de SP die stemt niet voor, even kijken ja de rest wel dus dat is allen met stemmen van de SP tegen is het aangenomen. Dan gaan wij naar agenda punt 9. Dat is het oneigenlijk gebruik van gemeentegrond. ik heb begrepen dat er wat punten zijn aangepast na de behandleing in de commisie maar dat de wethouder nog even een toevoeging heeft op die aapassingen. de heer bottigen. Meneer de voorzitter er is in de stukken een fout geslopen in het collegebesluit wat als raadsbesluit staat er wordt wel gesproken over dat het gaat over 2,2 miljoen verdeeld over 10 jaar maar dat dat per jaar wordt vastgesteld maar in het raadsbesluit is dat verkeerd opgenomen. Dus de tekst van het stuk van het raadsbesluit het is zo dat eigenlijk daarmee die motie die is ingediend op dit vlak of het abonnement waarbij staat dat er gedurende het bedrag nu van de kader noten van de vier jaar en dat dan per jaar wordt gekeken of dat bedrag ingezet kan worden, dat is ook de zeg maar de formulering zoals het college die heeft gedaan. Alleen die is in het raadsvoorstel onvoldoende of niet verwerkt. Dus we zijn het eens met elkaar. Waarbij wel de kanttekening overigens dat het geen zin heeft  om bijvoorbeeld volgend jaar al te zeggen van we hebben het een jaartje geprobeerd en het werkt niet."
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

# Haarlem - 2019 - 622012.mp4 - 1:46:45
ref = "Dank u wel. De heer Visser, Christen Unie. Dank u wel, voorzitter. Wat betreft de oude stukken die opnieuw geheim verklaaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op. Zo snel mogelijk in het najaar gaan we er opnieuw over hebben. Wat betreft het PWC-rapport is niet vanaf vandaag het openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft dateterrein ben ik toch een beetje benieuwd naar de reactie van het college. En ik heb inderdaad ook het gevoel dat we vanavond toch al op het zuid gaan komen. Dus waarom zouden we dat nu nog geheim verklaaren? Ik ben er niet van overtuigd wat daar het argument voor is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. Het is tot en met een ding niet akkoord. Al was het maar voor de symboolwerking. Maar bijlagen zeven bij de gedemd oudergeracht de uitgiftetekening. Ik heb net al gezegd ik vind het compleet een nonsens om zo'n tekening geheim te verklaren. Het college mag me wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. De heer van de Raad. Ja, voorzitter. Wij gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd. Dat hebben we bij het seniorenconvental aangegeven dat we daar niet meer van gediend zijn. Voor de rest heeft u gevraagd om alles in een keer te behandelen. U heeft ook gezegd alles komt terug. Dus u behandelt alles in een keer en het feit dat het terugkomt vinden wij dat niks geheim verklaard hoeft te worden. Dank u."
hyp = "Dank u wel. De heer visser, christen unie. Dank u wel voorzitter, wat betreft de oude stukken die opnieuw geheim verklaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op. zo snel mogelijk in het najaar gaan we het er opnieuw over hebben. wat betreft het pwcrapport is in ieder geval vanaf vandaag openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft Dateterrein ben ik toch een beetje benieuwd naar de reactie van het college. En ik heb inderdaad ook het gevoel dat we vanavond toch al een besluit gaan komen dus waarom zouden we dat nu nog geheim verklaren? Ik ben er niet van overtuigd wat daar het argument van is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. En tenslotte ga ik met een ding niet akkoord, al was het maar voor de symboolwerking. Maar bijlage zeven bij de gedemde ouderdrachte uitgiftetekening. Ik heb het net al gezegd, ik vind het complete onzin om zo'n tekening geheim te verklaren. Het college mag me wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. De heer van de raad. Ja voorzitter we gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd dat hebben we bij het seniorenconferent al aangegeven dat we daar niet meer van gediend zijn. En voor de rest heeft u gevraaagd om alles in een keer te behandelen en u heeft ook gezegd alles komt terug, dus ik behandel alles in een keer, en het feit dat het terug komt vinden wij dat niks geheim verklaard hoeft te worden dank u."
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

# Haarlem - 2024 - 1159903.mp4 - 8:10
ref = "Maar dat heeft niks te maken op zich met het referendum. Dan kan het college bevestigen dat er altijd bezwaar en beroep mogelijk is. Dat klopt niet, want in een aantal gevallen is er helemaal geen verkeersbesluit voor nodig. Iets wat niet legaal is, als je dat op een gegeven moment gaat niet meer gedoogd, dan neem je geen nieuw verkeersbesluit, want het was al illegaal. Dat moet je wel heel duidelijk markeren, want we gaan daar nu op een andere manier mee om. Ik heb al gezegd dat gebeurt alleen als er voldoende parkeerplaatsen beschikbaar zijn om de mensen met een vergunning te laten parkeren. In die situaties waarbij gelegaliseerd parkeren wordt opgeheven, daar is wel een parkeerbesluit voor en dat zal natuurlijk de gewone procedure doorlopen. Het opheffen van parkeerplaatsen, daarvoor geldt ook het reguliere participatie- en inspraakproject als dat zou gebeuren. Dan de vraag over de website. We proberen te voorkomen dat je de website vol stopt met informatie, maar nu dit zozeer een issue is"
hyp = "Maar dat heeft niks te maken op zich, met het referendum. Dan, kan het college bevestigen dat er altijd bezwaer en beroep mogelijk is. Dat klopt niet want in een aantal gevallen is er helemaal geen verkeersbesluit voor nodig. Iets wat niet legaal is, als je dat op een gegeven moment niet meer gedoogd dan neem je geen nieuw verkeersbesluit want het was al illegaal. Dan moet je natuurlijk wel heel duidelijk markeren van we gaan daar nu op een andere manier mee om. ik heb al gezegd dat gebeurt alleen als daar ook de ruimte voor is, als er dus voldoende verkeersplaatsen beschikbaar zijn om de mensen met een vergunning ook te laten parkeren. in die situaties waarbij gelegaliseerd parkeren wordt opgeheven daar is wel een parkeerbesluit voor en dat zal natuurlijk de gewone procedure doorlopen en het opheffen van parkeerplaatsen daarvoor geldt ook het reguliere participatie- en inspraak project als dat zou gebeuren. dan de vraag over de website we proberen te voorkomen dat je de website vol stopt met informatie maar nu dit zozeer een issue is "
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

# Barendrecht - 2024 - 1203463.mp4 - 49:36
ref = "Dat is op hun conto volgens de reactie van de wethouder die we vandaag ontvingen, waarbij hij vooral aangeeft onderbouwing te missen en geen aanleiding ziet tot aanpassen. Om het verhaal van beide kanten scherp te krijgen hebben we vandaag nog even gebeld met stad, kraf om deze reactie te toetsen en zij geven aan dat ze meerdere malen met de ambtelijke organisatie gesproken hebben, herkennen het uitblijven van gegevens niet en baseerde hun uitspraak op berekeningen vanuit andere projecten die ze hebben gedaan. Hoe verklaart de wethouder deze communicatie van langs elkaar in plaats van met elkaar? Het komt ons over als een welis nietes discussie, waar we verschillende partijen zien met een mening die niet overeenkomt, wat het lastig maakt voor ons om in te schatten wat nu de waarheid is. Wat in ieder geval duidelijk is voor ons is dat de provincie en stad kraft zich niet kunnen vinden in de voorliggende normen. Inhoudelijk, wil ik eigenlijk nog een keer een standpunt delen wat we over tijdens de orientatie, orienteerende verhalen hebben gedeeld, is dat wij vooral willen pleiten voor impact normen in plaats van input normen. Daarmee bedoelen we impact normen die iets zeggen over de impact op de omgeving, zoals geluidsnormen, slagschaduw enzovoort in plaats van input normen, zoals tiphoogte en afstanden. Dat zegt hoogheid iets over de beeldkwaliteit, maar wordt nu vooral gekoppeld aan geluid en dat is volgens mij een onlogische koppeling. Geluidsnormen moeten we gewoon verwoorden in geluidsnormen. Op die manier geven we ruimte aan de markt om de optimale oplossing te genereren, ook ter gunste van die omgeving. De provincie onderstreept het ook, hoe ze de wethouder dat, impact normen in plaats van input normen. Tot slot beschrijft het voorstel ook een stukje over monitoring en evaluatie, uitgaande dat de ontwikkeling mogelijk is. Stel nu dat de komende tijd toch mogelijk of toch blijkt dat de ontwikkeling niet mogelijk is. Op welke wijze wordt dat geevalueerd en welke vervolgacties worden dan getroffen. Tot zover, dank u wel. Voorzitter. Meneer Bol van Echt-Van Baarneet. Ik heb nu dit keurig even de bijdrage van meneer Koster van GroenLinks afgewacht, maar wat mij opviel u nu betogen is dat u aangeeft, ja ik heb vanmiddag even met de commerciele partij StadKraft gebeld en ze hebben u kennelijk verteld dat, nou wat ons college vertelt, dat dat niet klopt. U knikt ja voor mensen die mij alleen maar horen en u niet zien. Ja ik vind dat toch wel bijzonder en u noemt daar ook het woord vertrouwen in. Dus mijn vraag aan u is, vertrouwt u dan niet dat wat het college ons vanmiddag in een brief heeft geschreven dat dat klopt? En de tweede vraag eigenlijk is, ziet u dan niet in dat StadKraft, wat ik al zei een commerciele partij, dat die ook een bepaald belang heeft om te zorgen dat die winterpunis zo hoog en in aantal ook zoveel mogelijk daar komt. Ziet u ook niet in dat dat misschien ook wellicht gekleurd dan is hetgene wat ze tegen u gezegd hebben?"
hyp = "Dat is op hun conto volgens de reactie van de wethouder die we vandaag ontvingen waarbij hij vooral aangeeft onderbouwing te missen en geen aanleiding ziet tot aanpassen. Nou om het verhaal van beide kanten toch scherp te krijgen hebben we vandaag nog even gebeld met stadskraft om deze reactie te toetsen en zij geven aan dat ze meerdere malen met de ambtelijke organisatie gesproken hebben herkennen het uitblijven van gegevens niet en baseerde hun uitspraak op berekeningen vanuit andere projecten die ze hebben gedaan. Hoe verklaart de wethouder deze communicatie van langs elkaar in plaats van met elkaar? Het komt ons over als een weles nietes discussie waar we verschillende partijen zien met een mening die niet overeenkomt. Wat het lastig maakt voor ons om in te schatten wat nu de waarheid is. Wat in ieder geval duidelijk is voor ons is dat de provincie en stadskraft zich niet kunnen vinden in de voorliggende normen. Inhoudelijk wil ik eigenlijk nog een keer een standpunt delen wat we over tijdens de orientatie orienteerende vergadering hebben gedeeld, is dat wij vooral willen pleiten voor impact normen in plaats van input normen. Daarmee bedoelen we impact normen die iets zeggen over de impact op de omgeving zoals geluidsnormen, slagschaduw enzovoort in plaats van input normen zoals tiphoogte en afstanden. Dat zegt hooguit iets over de beeldkwaliteit maar wordt nu vooral gekoppeld aan geluid en dat is volgens mij een onlogische koppeling, geluidsnormen moeten we gewoon verwoorden in geluidsnormen. Op die manier geven we ruimte aan de markt om de optimale oplossing te genereren, ook ter gunste van de omgeving. De provincie onderstreept het ook. Hoe ze de wethouder dat? impact normen in plaats van input normen. Tot slot schrijft, beschrijft het voorstel ook een stukje over monitoring en evaluatie, uitgaande dat de ontwikkeling mogelijk is. Stel nu dat de komende tijd toch mogelijk, of toch blijkt dat de ontwikkeling niet mogelijk is. Op welke wijze wordt dat geevalueerd en welke vervolgacties worden dan getroffen? Tot zover, dank u wel. Voorzitter. Meneer Bol van Echt Van Baarnet. Ik heb nu dit keurig even de bijdrage van meneer Koster van GroenLinks afgewacht, maar wat mij opviel in uwbetoog is dat u aangeeft van ja ik heb vanmiddag even met de commerciele partij StadsKraft gebeld en ze hebben u kennelijk verteld dat, nou wat ons college vertelt, dat dat niet klopt, als ik dat zo mag u knikt ja voor mensen die mij alleen maar horen en u niet zien. Ja ik vind dat toch wel bijzonder en u noemt daar ook het woord vertrouwen in. Dus mijn vraag aan u is, vertrouwt u dan niet dat wat het college ons vanmiddag in een brief heeft geschreven dat dat klopt? En de tweede vraag eigenlijk is, ziet u dan niet in dat StadsKraft, wat ik al zei een commerciele partij, dat die ook een bepaald belang heeft om te zorgen dat die winterpuzzel hoog en in aantal ook zoveel mogelijk daar komt. Ziet u ook niet in dat dat misschien ook wellicht gekleurd dan is hetgene wat ze tegen u gezegd hebben?"
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

# Hoekschewaard - 2023 - 1068492.mp4 - 1:29:25
ref = "dus het lijkt me goed om daar schriftelijk op terug te komen, om ook nog weleens aan te geven wat we er allemaal aan doen. Oke, maar misschien kan dat ook meegenomen worden naar de volgende week, of die kan voor volgende week dan voor de sluitvorming? Technische vragen hoeven ook niet langs het college, dus die kunnen heel snel, die kunnen extra snel. Ja, oke, goed. Ik heb verder, ik moet vragen, de heer Rayen. Ja, dank u wel. Ik vroeg me even af hoe de wethouder aankijkt tegen de uitvoerbaarheid, want er zijn 900 huishoudens om en nabij in de categorie 120 tot 130 procent, van sociaal minimum. Potentieel, misschien gaan het niet iedereen bijzonder bijzonder aanvragen, maar toch zeker wel een aanzienlijk deel. En hoe ziet dat er qua uitvoering uit? En moeten dan niet gewoon geld bij in de uitvoering? En mist dat niet dan in het voorstel? Dank u wel. De weghouder. Voorzitter, de suggestie of de opmerking die de heer Rayen maakt van zeker wel een aanzienlijk deel, dat is nog maar de vraag. Als je kijkt naar waar die energie toetslag, als je kijkt wat de energiekosten op dit moment zijn voor huishouden, het is in opzichte van de periode daarvoor, dan is het nog maar de vraag of heel veel mensen een aanvraag doen als het gaat over de kosten van energie. Daarnaast heeft het natuurlijk ook nog een vermogenstoets, die zit er bij die 120 procent niet bij. Dus op het moment dat je een miljoen euro op de bank hebt, maar 120 procent van het minimum krijgt, krijg je toch wel de energietoetslag, want mij gaat er dan maar vanuit dat als je weinig inkomsten hebt, ook geen vermogen hebt. Dat staat ook zo in het voorstel, dat staat ook zo in de wet genoemd. En bij die 130 procent wordt er dan wel gekeken op de vermogen. Wij denken dat die aanpak op zich binnen de huidige formatie, plus die mogelijke inhuur als dat nodig is, binnen die acht weken te realiseren is. Dat is eigenlijk de vraag die je gesteld hebt. De heer A.M. tevreden? Ja. Mevrouw Verbrug? Ja, bij de vermogenstoets kijken ze ook altijd om mensen een auto hebben. En veel mensen hebben een auto gewoon nodig om naar de werk te komen. En als mensen dan toch die energietoetslag aanvragen en ze hebben een auto, moeten ze die dan eerst verkopen zodat ze een hele oude terug kunnen kopen, voordat ze uberhaupt een vergoeding kunnen krijgen? Ik weet niet of we de weghouden die kan antwoorden, anders moeten we een technisch schriftelijk afdoen. Ja, dit gaat erg, de techniek, en het lijkt wel goed om daar gewoon een goed onderbouwtechnisch antwoord op te geven. Oke. Dan kunnen we voorzorgden dat we daarvoor, bij de besluit van de vergadering, het antwoord hebben daarop. En dat geldt ook nog even terug, want we hebben de vraag van meneer Raaijer. Goed, dan kijk ik verder, heer de Koning. Ja, dank u wel voorzitter. Ik had natuurlijk niet kunnen vermoeden dat ik de vergelijking met een kruidenuur maakte, dat dat eerder als een geuzennaam werd opgepakt als de wethoud. Dus ik moet nu eigenlijk iets anders verzinnen. Nee, laten we het eventjes over het andere aspect. Ik had het over een man, een man, een woord, een woord, en de kruidenuursgeest daar tegenover. En misschien is er niks te naderder van de kruidenuurs."
hyp = "dus het lijkt me goed om daar schriftelijk op terug te komen om ook nog weereens aan te geven wat we er allemaal aan doen. Oke maar misschien kan dat ook meegenomen worden naar de volgende week, of die kan voor volgende week dan voor de besluitvorming? Technische vragen hoeven ook niet langs het college, dus die kunnen heel snel, die kunnen extra snel. Ja oke goed. Ik heb verder, ik moet vragen, de heer Raaijer. Ja, dank u wel. Ik vroeg me even af hoe de wethouder aankijkt tegen de uitvoerbaarheid want er zijn 900 huishoudens om en nabij in de categorie 120 tot 130 procent, van sociaal minimum. Potentieel, nou ja misschien gaat niet iedereen bijzonder bijstand aanvragen, maar toch zeker wel een aanzienlijk deel. En hoe ziet dat er qua uitvoering uit? En moeten dan niet gewoon geld bij in de uitvoering? En mist dat dan niet in het voorstel? Dank u wel. De wethouder. Voorzitter, de suggestie of de opmerking die de heer Raaijer maakt van zeker wel een aanzienlijk deel, dat is nog maar de vraag. Als je kijkt naar waar die energie toeslag, als je kijkt wat de energiekosten op dit moment zijn voor huishoudens, ten opzichte van de periode daarvoor, dan is het nog maar de vraag of heel veel mensen een aanvraag doen als het gaat over de kosten van energie. Daarnaast heeft het natuurlijk ook nog een vermogenstoets, die zit er bij die 120 procent niet bij. Dus op het moment dat je een miljoen euro op de bank hebt, maar 120 procent van het minimum krijgt, krijg je toch wel de energietoetslag, want men gaat er dan maar vanuit dat als je weinig inkomsten hebt, ook geen vermogen hebt. Dat staat ook zo in het voorstel, dat staat ook zo in de wet genoemd. En bij die 130 procent wordt er dan wel gekeken op de vermogen. Wij denken dat die aanpak op zich binnen de huidige formatie, plus die mogelijke inhuur als dat nodig is, binnen die acht weken te realiseren is. Dat is eigenlijk de vraag die u gesteld hebt. De heer Raaijer tevreden? Ja. Mevrouw de brug? Ja, bij de vermogenstoets kijken ze ook altijd om mensen een auto hebben. En veel mensen hebben een auto gewoon nodig om naar hun werk te komen. En als mensen dan toch die energietoeslag aanvragen en ze hebben een auto, moeten ze die dan eerst verkopen zodat ze een hele oude terug kunnen kopen, voordat ze uberhaupt een vergoeding kunnen krijgen? Ik weet niet of we de wethouder dit kan antwoorden, anders moeten we het technisch schriftelijk afdoen. Ja, dit gaat erg, de techniek, en het lijkt wel goed om daar gewoon een goed onderbouw technisch antwoord op te geven. Oke dan kunnen we ervoor zorgen dat we daarvoor, bij de besluit van de vergadering, het antwoord hebben daarop. En dat geldt ook nog even terug, want we hebben de vraag van meneer Raaijer. Goed, dan kijk ik verder, de heer de Koning. Ja, dank u wel voorzitter ja ik had natuurlijk niet kunnen vermoeden dat ik de vergelijking met een kruidenier maakte dat dat eerder als een geuzennaam werd opgepakt als de wethouder. Dus ik moet nu eigenlijk iets anders verzinnen maar Nee, laten we het eventjes over het andere aspect. Ik had het over een man een man een woord een woord, en de kruideniersgeest daar tegenover. En misschien is er niks te naderder van de kruideniers."
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

WER: 0.07
WER: 0.12
WER: 0.11
WER: 0.05
WER: 0.07


In [151]:
# Calculates accurateness of pyannote speaker diarisation

# Calculates the overlapping duration between two time segments.
def calculate_overlap(segment1, segment2):
    start1, end1 = segment1
    start2, end2 = segment2
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)
    overlap = max(0, overlap_end - overlap_start)
    return overlap


# Calculates the Diarisation Error Rate (DER).
def der(ground_truth, hypothesis, start_time, end_time):
    total_time = 0
    missed_speech = 0
    false_alarm = 0
    speaker_error = 0

    for gt_start, gt_end, gt_speaker in ground_truth:
        total_time += gt_end - gt_start
        matched = False
        for hyp_start, hyp_end, hyp_speaker in hypothesis:
            overlap = calculate_overlap((gt_start, gt_end), (hyp_start, hyp_end))
            if overlap > 0:
                matched = True
                if gt_speaker != hyp_speaker:
                    speaker_error += overlap
        if not matched:
            missed_speech += gt_end - gt_start

    for hyp_start, hyp_end, hyp_speaker in hypothesis:
        matched = False
        for gt_start, gt_end, gt_speaker in ground_truth:
            overlap = calculate_overlap((gt_start, gt_end), (hyp_start, hyp_end))
            if overlap > 0:
                matched = True
        if not matched:
            false_alarm += hyp_end - hyp_start

    der = (missed_speech + false_alarm + speaker_error) / (end_time - start_time)

    return der


In [152]:
START = 3
DUR = 4
SPEAKER = 7


def round_to_nearest_interval(value, interval=0.125):
    return round(value / interval) * interval


def get_ref(filepath, start, end):
    reference = Annotation()
    with open(filepath, "r") as f:
        for line in f:
            info = line.strip().replace("\n", "").split()
            start_segm = float(info[START])
            end_segm = float(info[START]) + float(info[DUR])
            if start_segm >= start and end_segm <= end:
                reference[
                    Segment(
                        (start_segm),
                        (end_segm),
                    )
                ] = info[SPEAKER]
    # print(reference)
    # print("----")
    return reference


def get_hyp(filepath, start, end):
    data_path = "diar_data.json"
    with open(data_path, "r") as f:
        data = json.load(f)

    hypothesis = Annotation()
    for entry in data:
        if entry["file"] == filepath:
            for e in entry["data"]:
                start_segm = e[0]
                end_segm = e[1]
                if start_segm >= start and end_segm <= end:
                    hypothesis[
                        Segment(
                            (start_segm), (end_segm)
                        )
                    ] = e[2]

            # print(hypothesis)
            return hypothesis

    print(f"{filepath} not found in {data_path}")

In [153]:
def der(filepath, start, end, collar=0.0):
    diarizationErrorRate = DiarizationErrorRate(
        collar=collar,
    )
    metrics = diarizationErrorRate(
        get_ref(filepath, start, end),
        get_hyp(filepath, start, end),
        uem=Segment(start, end),
        detailed=True,
    )

    total_time = end - start
    false_alarm = metrics["false alarm"]
    missed = metrics["missed detection"]
    confusion = metrics["confusion"]
    der = metrics["diarization error rate"]

    return total_time, false_alarm, missed, confusion, der

In [172]:
filepath = "/Users/personal/Desktop/scriptie/notebooks/data/nijmegen/100000013/2022/diorizations/80ec1e6a-8c74-4f41-bcc5-6f5f27d48512.wav.rttm"
start = 3208
end = 3499
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.250))
print(der(filepath, start, end, collar=0.500), "\n")

filepath = "/Volumes/Samsung_T5/data/hoekschewaard/vergaderingen/2023/diorizations/1068492.wav.rttm"
start = 3533
end = 3798
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.250))
print(der(filepath, start, end, collar=0.500), "\n")

filepath = "/Volumes/Samsung_T5/data/ridderkerk/vergaderingen/2022/diorizations/986111.wav.rttm"
start = 3533
end = 3877
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.250))
print(der(filepath, start, end, collar=0.500), "\n")

filepath = "/Volumes/Samsung_T5/data/ridderkerk/vergaderingen/2022/diorizations/938576.wav.rttm"
start = 2320
end = 2649
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.250))
print(der(filepath, start, end, collar=0.500), "\n")

filepath = "/Volumes/Samsung_T5/data/barendrecht/vergaderingen/2024/diorizations/1203463.wav.rttm"
start = 2516
end = 2977
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.250))
print(der(filepath, start, end, collar=0.500), "\n")

(291, 4.007000000000062, 8.826999999999316, 0.3220000000001164, 0.05435173288494472)
(291, 0.6289999999989959, 5.864999999999327, 0.07200000000011642, 0.028760277002722025)
(291, 0.09399999999959618, 5.257999999999811, 0.0, 0.02492443964252342) 

(265, 1.829000000000633, 24.08299999999963, 3.5280000000002474, 0.13595703314414714)
(265, 0.3739999999997963, 18.496999999999844, 2.5390000000006694, 0.10526005280210118)
(265, 0.0009999999997489795, 15.54699999999957, 1.6030000000005202, 0.08968827949735518) 

(344, 27.335999999999785, 31.860999999998967, 3.5650000000000546, 0.21522433919042697)
(344, 26.35199999999986, 30.42199999999866, 3.3150000000000546, 0.2094700587739002)
(344, 25.786000000000058, 30.034999999998945, 3.0650000000000546, 0.2086018229544864) 

(329, 7.095000000000709, 6.445999999997639, 0.3890000000001237, 0.053202866003630236)
(329, 0.941000000000713, 3.774999999999636, 0.1570000000001528, 0.020359136334772657)
(329, 0.1340000000004693, 3.1719999999995707, 0.0, 0.015165